In [11]:
!pip install langchain pinecone-client tiktoken petals

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 15.3 MB/s eta 0:00:00


In [12]:
import os
import getpass

os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")
os.environ["PINECONE_ENV"] = getpass.getpass("Pinecone Environment:")
os.environ["HUGGINGFACE_API_KEY"] = getpass.getpass("Huggingface API Key:")

Pinecone API Key:··········
Pinecone Environment:··········
Huggingface API Key:··········


In [15]:
from langchain.llms import Petals
from langchain import PromptTemplate, LLMChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import Pinecone

In [17]:
def csv_loader(file_path='./data/data.csv'):
    loader = CSVLoader(
        file_path,
        csv_args={
            'delimiter': ';',
            'quotechar': '"',
            'fieldnames': ['price', 'description', 'slug'],
        },
    )
    return loader.load()

In [21]:
import pinecone

# initialize pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV"),
)

In [29]:
!pip install InstructorEmbedding sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=eec5d7c26033d358877992cb906f036556e8fdc559e0989de129e99ad246a38a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [30]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

embeddings = HuggingFaceInstructEmbeddings(
    model_name='hkunlp/instructor-large',
    encode_kwargs={'normalize_embeddings': True},
    query_instruction="Represent the query for retrieval: "
)


Aug 04 14:52:52.712 [INFO] Load pretrained SentenceTransformer: hkunlp/instructor-large


load INSTRUCTOR_Transformer


Aug 04 14:53:31.351 [INFO] Use pytorch device: cpu


max_seq_length  512


In [34]:
docs = csv_loader()
docsearch = Pinecone.from_documents(docs, embeddings, index_name='llama-2')

In [87]:
question = "how much is the price of honda brio in depok city?"
results = docsearch.similarity_search_with_score(question, k=2)

for doc in  results:
    content, score = doc
    print(f'Content: {content}')
    print(f'Score: {score}')

Content: page_content='price: 137,000,000\ndescription: Honda Brio 1.2 Satya E City Car Abu-abu Otomatis Bensin 2018 Kota Depok Jawa Barat\nslug: honda-12-satya-e-brio-2018-20' metadata={'row': 20.0, 'source': './data/data.csv'}
Score: 0.917420685
Content: page_content='price: 143,000,000\ndescription: Honda Brio 1.2 RS CVT City Car Abu-abu Tua Metalik Manual Bensin 2018 Kota Depok Jawa Barat\nslug: honda-12-rs-cvt-brio-2018-09' metadata={'row': 9.0, 'source': './data/data.csv'}
Score: 0.917057276


In [44]:
llm = Petals(model_name="meta-llama/Llama-2-70b-chat-hf")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Aug 04 15:09:37.199 [INFO] Make sure you follow the LLaMA's terms of use: https://bit.ly/llama2-license for LLaMA 2, https://bit.ly/llama-license for LLaMA 1
Aug 04 15:09:37.200 [INFO] Using DHT prefix: Llama-2-70b-chat-hf


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [88]:
template = """
Instructions:
Generate an answer to the user's question based on the provided search results.

Context:
Include as much information from localhost as possible in the answer, referencing the relevant search result.
{context}

Input Data:
Question: {question}

Output Indicator:
1. The output should include detailed information about cars.
2. The output should be numbered if the desired car exists.
3. The output should be divided into sections for Price, Description, and Link if the desired car exists.
4. The link output must be combined with a slug using the following format: http://localhost/slug.
5. The output should be formatted in markdown."""

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=['context','question'],
    template=template,
)

In [89]:
from langchain.chains import RetrievalQA

retriever = docsearch.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=docsearch.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

ans = qa_chain({'query': question})
print(ans['result'])


Aug 04 16:14:57.516 [INFO] Route found: 0:40 via …uCst2v => 40:80 via …o6L7CF


<s> 
Instructions:
Generate an answer to the user's question based on the provided search results.

Context:
Include as much information from localhost as possible in the answer, referencing the relevant search result.
price: 137,000,000
description: Honda Brio 1.2 Satya E City Car Abu-abu Otomatis Bensin 2018 Kota Depok Jawa Barat
slug: honda-12-satya-e-brio-2018-20

price: 143,000,000
description: Honda Brio 1.2 RS CVT City Car Abu-abu Tua Metalik Manual Bensin 2018 Kota Depok Jawa Barat
slug: honda-12-rs-cvt-brio-2018-09

price: 106,000,000
description: Honda Brio 1.2 Satya E City Car Merah Metalik Manual Bensin 2015 Kota Depok Jawa Barat
slug: honda-12-satya-e-brio-2015-15

price: 173,000,000
description: Honda Brio 1.2 RS CKD Mini Bus Abu-abu Metalik Manual Bensin 2021 Kota Jakarta Selatan Dki Jakarta
slug: honda-12-rs-ckd-brio-2021-10

Input Data:
Question: how much is the price of honda brio in depok city?

Output Indicator:
1. The output should include detailed information abou